# Edge detection using the gradient magnitude and threshold segmentation

## Compiler parameters

Set my Jypyter environment for the use of OpenCV in a C++ notebook. You don't need this line when you write your own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use CMake.

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <stdexcept>
#include <sstream>
#include <string>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Read the image from a file on the disk and return a new matrix

![../cells_greyscale.png](../cells_greyscale.png)

In [5]:
Mat image = imread("../cells_greyscale.png", IMREAD_GRAYSCALE);

## Check for errors

In [6]:
// Check for failure
if (image.empty()) 
{
    throw runtime_error("Could not open or find the image");
}

## Compute the derivative along the two directions

See [https://docs.opencv.org/master/d2/d2c/tutorial_sobel_derivatives.html](https://docs.opencv.org/master/d2/d2c/tutorial_sobel_derivatives.html) or details of the functions.

In [7]:
Mat grad_x, grad_y;
Mat abs_grad_x, abs_grad_y;

int ksize = 3; // Use a 3x3 Sobel kernel

Sobel(image, grad_x, CV_32F, 1, 0, ksize); // dx = 1, dy = 0
Sobel(image, grad_y, CV_32F, 0, 1, ksize); // dx = 0, dy = 1

## Show the images

In [8]:
namedWindow("grad_x", WINDOW_GUI_EXPANDED);
imshow("grad_x", grad_x);

namedWindow("grad_y", WINDOW_GUI_EXPANDED);
imshow("grad_y", grad_y);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | gard_x | grad_y |
|----------------|--------|--------|
|![image](../image.png) | ![grad_x](../grad_x.png) | ![grad_y](../grad_y.png) |

Not very good. What's wrong? We must normalise the data first. See the min and max pixel values?

In [9]:
double min_value, max_value;

cv::minMaxLoc(grad_x, &min_value, &max_value);
cout << "grad_x, min: " << min_value << "   max: " << max_value << endl;

cv::minMaxLoc(grad_y, &min_value, &max_value);
cout << "grad_y, min: " << min_value << "   max: " << max_value << endl;

grad_x, min: -268   max: 257
grad_y, min: -278   max: 255


Within the range 0.0 and 1.0 (because of using CV_32F (float32))? NO! We must normalise the image for the visualisation.

In [10]:
Mat vis_grad_x;
Mat vis_grad_y;

cv::normalize(grad_x, vis_grad_x, 0, 255, NORM_MINMAX, CV_8UC1);
cv::normalize(grad_y, vis_grad_y, 0, 255, NORM_MINMAX, CV_8UC1);

In [11]:
namedWindow("grad_x", WINDOW_GUI_EXPANDED);
imshow("grad_x", vis_grad_x);

namedWindow("grad_y", WINDOW_GUI_EXPANDED);
imshow("grad_y", vis_grad_y);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | gard_x | grad_y |
|----------------|--------|--------|
|![image](../image.png) | ![grad_x](../vis_grad_x.png) | ![grad_y](../vis_grad_y.png) |

Better? Same data but presented differently

## Compute the gradient magnitude

In [12]:
Mat abs_grad_x;
Mat abs_grad_y;

abs_grad_x = abs(grad_x);
abs_grad_y = abs(grad_y);

### Visualise the result

In [13]:
Mat vis_abs_grad_x;
Mat vis_abs_grad_y;

cv::normalize(abs_grad_x, vis_abs_grad_x, 0, 255, NORM_MINMAX, CV_8UC1);
cv::normalize(abs_grad_y, vis_abs_grad_y, 0, 255, NORM_MINMAX, CV_8UC1);

In [14]:
namedWindow("abs_grad_x", WINDOW_GUI_EXPANDED);
imshow("abs_grad_x", vis_abs_grad_x);

namedWindow("abs_grad_y", WINDOW_GUI_EXPANDED);
imshow("abs_grad_y", vis_abs_grad_y);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | abs_grad_x | abs_grad_y |
|----------------|--------|--------|
|![image](../image.png) | ![grad_x](../vis_abs_grad_x.png) | ![grad_y](../vis_abs_grad_y.png) |

### Combine the gradient along the two directions

In [15]:
Mat grad;
addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0, grad);

In [16]:
Mat vis_grad;

cv::normalize(grad, vis_grad, 0, 255, NORM_MINMAX, CV_8UC1);

In [17]:
namedWindow("grad", WINDOW_GUI_EXPANDED);
imshow("grad", vis_grad);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | grad |abs_grad_x | abs_grad_y |
|----------------|------|--------|--------|
|![image](../image.png) | ![grad](../vis_grad.png) | ![grad_x](../vis_abs_grad_x.png) | ![grad_y](../vis_abs_grad_y.png) |

## Segmentation using the threshold technique

See [https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57) or details of the functions. 

See here for the threshold types: [https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576](https://docs.opencv.org/master/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576)

In [18]:
Mat edge;
cv::threshold(vis_grad, edge, 64, 255, THRESH_BINARY);

In [19]:
namedWindow("grad", WINDOW_GUI_EXPANDED);
imshow("grad", vis_grad);

namedWindow("edge", WINDOW_GUI_EXPANDED);
imshow("edge", edge);

namedWindow("image", WINDOW_GUI_EXPANDED);
imshow("image", image);

waitKey(0); // Wait for any keystroke in the window
destroyAllWindows(); // Destroy all the created windows

| Original image | grad | edge |
|----------------|--------|--------|
|![image](../image.png) | ![grad](../vis_grad.png) | ![edge](../edge.png) |